# 18차시: [실습] 매일 아침 주요 경제 지표 자동 수집 및 리포트 발송

## 학습 목표
- 모듈 2에서 배운 내용을 종합하여 자동화 파이프라인 구축
- 주요 금융 지표 수집 → 리포트 생성 → 이메일 발송

## 학습 내용
1. 전체 파이프라인 설계
2. 데이터 수집 통합 (15-16차시 함수 재사용)
3. FRED 경제지표 수집 (13차시 함수 재사용)
4. 리포트 생성
5. Gmail 이메일 발송
6. 전체 파이프라인 실행

---
## 1. 전체 파이프라인 설계

네이버 금융(환율, 유가, 뉴스) / FRED API (금리) -->  SQLite  저장 --> 리포트 생성 (텍스트/HTML) -->  Gmail  발송   

### 수집할 데이터
| 데이터 | 출처 | 기존 차시 |
|--------|------|-----------|
| 환율 (USD, EUR, JPY, CNY) | 네이버 금융 | 15-16차시 |
| 유가/금 시세 | 네이버 금융 | 15-16차시 |
| 주요 뉴스 헤드라인 (Top 5) | 네이버 금융 | 15차시 |
| 미국 기준금리 (FEDFUNDS) | FRED API | 13차시 |
| 10년 국채 수익률 (DGS10) | FRED API | 13차시 |

In [1]:
# 기본 라이브러리 import
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import os
from IPython.display import display

---
## 2. 데이터 수집 통합 함수

15-16차시에서 배운 크롤링 함수를 재사용합니다.

In [2]:
# 공통 설정
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

def get_soup(url):
    """URL에서 BeautifulSoup 객체 반환"""
    response = requests.get(url, headers=HEADERS, timeout=10)
    response.raise_for_status()
    return BeautifulSoup(response.text, 'html.parser')

In [3]:
# 시장 지표 크롤링 함수 (15차시 복습)
def crawl_market_indicators():
    """네이버 금융에서 환율, 유가, 금 시세 크롤링"""
    url = "https://finance.naver.com/marketindex/"
    soup = get_soup(url)

    data = []

    def get_direction(item):
        """등락 방향 확인"""
        if item.select_one('.point_up'):
            return '상승'
        elif item.select_one('.point_dn'):
            return '하락'
        return '보합'

    # 환율
    exchange_list = soup.select('#exchangeList li')
    for item in exchange_list:
        name = item.select_one('.h_lst .blind')
        value = item.select_one('.head_info .value')
        change = item.select_one('.change')

        if name and value:
            data.append({
                '분류': '환율',
                '지표명': name.get_text(strip=True),
                '현재가': value.get_text(strip=True),
                '등락': change.get_text(strip=True) if change else '',
                '등락방향': get_direction(item)
            })

    # 유가/금
    oil_gold_list = soup.select('#oilGoldList li')
    for item in oil_gold_list:
        name = item.select_one('.h_lst .blind')
        value = item.select_one('.head_info .value')
        change = item.select_one('.change')

        if name and value:
            data.append({
                '분류': '원자재',
                '지표명': name.get_text(strip=True),
                '현재가': value.get_text(strip=True),
                '등락': change.get_text(strip=True) if change else '',
                '등락방향': get_direction(item)
            })

    return pd.DataFrame(data) if data else pd.DataFrame()

In [4]:
# 뉴스 크롤링 함수 (15차시 복습)
def crawl_financial_news(limit=5):
    """네이버 금융 주요 뉴스 크롤링"""
    url = "https://finance.naver.com/news/mainnews.naver"
    soup = get_soup(url)

    news_data = []
    news_items = soup.select('ul.newsList li')[:limit]

    for item in news_items:
        title_tag = item.select_one('dd.articleSubject a')
        summary_tag = item.select_one('dd.articleSummary')
        press_tag = item.select_one('.press')

        if title_tag:
            title = title_tag.get_text(strip=True)
            link = 'https://finance.naver.com' + title_tag.get('href', '')
            summary = ''
            press = ''

            if summary_tag:
                summary_text = summary_tag.get_text(strip=True)
                # 출처 부분 제거
                if press_tag:
                    press = press_tag.get_text(strip=True)
                    summary = summary_text.replace(press, '').strip()

            news_data.append({
                '제목': title,
                '요약': summary[:100] + '...' if len(summary) > 100 else summary,
                '출처': press,
                '링크': link
            })

    return pd.DataFrame(news_data) if news_data else pd.DataFrame()

In [5]:
# 크롤링 함수 테스트
print("[시장 지표 크롤링 테스트]")
print("=" * 60)
df_market = crawl_market_indicators()
display(df_market.head())

print("\n[뉴스 크롤링 테스트]")
print("=" * 60)
df_news = crawl_financial_news(limit=5)
display(df_news[['제목', '출처']])

[시장 지표 크롤링 테스트]


,분류,지표명,현재가,등락,등락방향
0,환율,미국 USD,"1,458.40",5.40,상승
1,환율,일본 JPY(100엔),926.64,0.31,상승
2,환율,유럽연합 EUR,"1,699.40",5.13,상승
3,환율,중국 CNY,208.91,0.81,상승
4,원자재,WTI,57.76,1.77,상승



[뉴스 크롤링 테스트]


,제목,출처
0,"5000피 넘어 6000피 부채질?…'국장 장투' ISA 세제혜택, 효과는",머니투데이
1,트럼프 한마디에 이틀 만에 20만원 올랐다…52주 신고가 찍은 한화에어로,매일경제
2,국장 특화 '新ISA' 나온다…'역대급' 세혜택 부여,한국경제TV
3,"최고가 찍었는데 '사라, 마라' 보고서가 없다...한미반도체만 왜?",머니투데이
4,"Private Equity 전략의 핵심, 바이아웃[마켓칼럼]",한국경제


---
## 3. FRED 경제지표 수집

13차시에서 배운 FRED API를 활용합니다.

In [6]:
# API 키 로드 (11차시 방식)
from dotenv import load_dotenv

# .env 파일 업로드 (Colab)
try:
    from google.colab import files
    print("[Colab 환경] .env 파일을 업로드해주세요.")
    print("파일에 FRED_API_KEY=your_api_key 형식으로 저장되어 있어야 합니다.")
    uploaded = files.upload()
    load_dotenv('.env')
except ImportError:
    # 로컬 환경
    load_dotenv()

FRED_API_KEY = os.getenv('FRED_API_KEY')

if FRED_API_KEY:
    print(f"FRED API 키 로드 완료: {FRED_API_KEY[:5]}...")
else:
    print("[경고] FRED API 키가 없습니다. FRED 데이터 수집을 건너뜁니다.")

FRED API 키 로드 완료: 06171...


In [7]:
# FRED 데이터 수집 함수 (13차시 복습)
def fetch_fred_series(series_id, api_key):
    """FRED API에서 단일 시리즈 데이터 조회 (최근 값만)"""
    base_url = "https://api.stlouisfed.org/fred/series/observations"

    params = {
        'series_id': series_id,
        'api_key': api_key,
        'file_type': 'json',
        'sort_order': 'desc',
        'limit': 1  # 최근 1개만
    }

    try:
        response = requests.get(base_url, params=params, timeout=10)
        response.raise_for_status()

        data = response.json()
        observations = data.get('observations', [])

        if observations:
            obs = observations[0]
            return {
                'series_id': series_id,
                'date': obs['date'],
                'value': obs['value']
            }
    except Exception as e:
        print(f"  [오류] {series_id}: {e}")

    return None

In [8]:
# FRED 경제지표 수집
def collect_fred_indicators(api_key):
    """주요 FRED 경제지표 수집"""
    if not api_key:
        return pd.DataFrame()

    indicators = {
        'FEDFUNDS': '미국 기준금리',
        'DGS10': '미국 10년 국채'
    }

    data = []

    for series_id, name in indicators.items():
        result = fetch_fred_series(series_id, api_key)
        if result:
            data.append({
                '분류': '경제지표',
                '지표명': name,
                '현재가': f"{result['value']}%",
                '기준일': result['date']
            })
        time.sleep(0.3)  # API 호출 딜레이

    return pd.DataFrame(data) if data else pd.DataFrame()

In [9]:
# FRED 데이터 수집 테스트
print("[FRED 경제지표 수집]")
print("=" * 60)
df_fred = collect_fred_indicators(FRED_API_KEY)

if not df_fred.empty:
    display(df_fred)
else:
    print("FRED 데이터가 없습니다. (API 키 확인 필요)")

[FRED 경제지표 수집]


,분류,지표명,현재가,기준일
0,경제지표,미국 기준금리,3.72%,2025-12-01
1,경제지표,미국 10년 국채,4.15%,2026-01-07


---
## 4. 리포트 생성

수집한 데이터를 보기 좋은 텍스트 리포트로 변환합니다.

In [10]:
def generate_report(df_market, df_news, df_fred):
    """수집한 데이터로 텍스트 리포트 생성
    df_market : 환율, 원자재 등 시장 지표 데이터
    df_fred : 미국 경제지표(FRED) 데이터
    """
    today = datetime.now().strftime('%Y-%m-%d')

    lines = []    # 리포트 문장을 한 줄씩 담을 리스트
    lines.append(f"[오늘의 금융 지표 리포트] {today}")
    lines.append("=" * 50)

    # 환율
    lines.append("\n=== 환율 ===")
    # '분류'가 환율인 데이터만 필터링
    df_exchange = df_market[df_market['분류'] == '환율']
    # 각 환율 지표를 한 줄씩 리포트에 추가
    for _, row in df_exchange.iterrows():
        # 등락 정보가 있으면 방향과 값을 함께 표시
        direction = f"({row['등락방향']} {row['등락']})" if row['등락'] else ""
        lines.append(f"  {row['지표명']}: {row['현재가']} {direction}")

    # 원자재
    lines.append("\n=== 원자재 ===")
    # '분류'가 원자재인 데이터만 필터링
    df_commodity = df_market[df_market['분류'] == '원자재']
    for _, row in df_commodity.iterrows():
        direction = f"({row['등락방향']} {row['등락']})" if row['등락'] else ""
        lines.append(f"  {row['지표명']}: {row['현재가']} {direction}")

    # FRED 경제지표
    lines.append("\n=== 미국 경제지표 ===")
    for _, row in df_fred.iterrows():
        lines.append(f"  {row['지표명']}: {row['현재가']} (기준: {row['기준일']})")

    # 뉴스
    lines.append("\n=== 주요 뉴스 ===")
    for i, row in df_news.iterrows():
        # 제목이 너무 길 경우 40자까지만 표시
        title = row['제목'][:40] + '...' if len(row['제목']) > 40 else row['제목']
        lines.append(f"  {i+1}. {title}")
        lines.append(f"     - {row['출처']}")

    lines.append("\n" + "=" * 50)
    lines.append("이 리포트는 자동으로 생성되었습니다.")

    return "\n".join(lines)   # 리스트를 하나의 문자열로 결합하여 반환

In [11]:
# 리포트 생성 테스트
print("[리포트 생성 테스트]")
print("=" * 60)
report = generate_report(df_market, df_news, df_fred)
print(report)

[리포트 생성 테스트]
[오늘의 금융 지표 리포트] 2026-01-09

=== 환율 ===
  미국 USD: 1,458.40 (상승 5.40)
  일본 JPY(100엔): 926.64 (상승 0.31)
  유럽연합 EUR: 1,699.40 (상승 5.13)
  중국 CNY: 208.91 (상승 0.81)

=== 원자재 ===
  WTI: 57.76 (상승 1.77)
  휘발유: 1713.62 (하락 2.12)
  국제 금: 4460.7 (하락 1.80)
  국내 금: 209806.68 (상승 1,246.52)

=== 미국 경제지표 ===
  미국 기준금리: 3.72% (기준: 2025-12-01)
  미국 10년 국채: 4.15% (기준: 2026-01-07)

=== 주요 뉴스 ===
  1. 5000피 넘어 6000피 부채질?…'국장 장투' ISA 세제혜택, 효과...
     - 머니투데이
  2. 트럼프 한마디에 이틀 만에 20만원 올랐다…52주 신고가 찍은 한화에어로
     - 매일경제
  3. 국장 특화 '新ISA' 나온다…'역대급' 세혜택 부여
     - 한국경제TV
  4. 최고가 찍었는데 '사라, 마라' 보고서가 없다...한미반도체만 왜?
     - 머니투데이
  5. Private Equity 전략의 핵심, 바이아웃[마켓칼럼]
     - 한국경제

이 리포트는 자동으로 생성되었습니다.


---
## 5. Gmail 이메일 발송

Gmail SMTP를 사용하여 리포트를 이메일로 발송합니다.

### Gmail 앱 비밀번호 생성 방법
1. Google 계정 → 보안 → 2단계 인증 활성화
2. 앱 비밀번호 생성 (메일 → Windows 컴퓨터)
3. 생성된 16자리 비밀번호를 `.env`에 저장

In [12]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def send_email_gmail(subject, body, to_email, sender_email, app_password):
    """Gmail SMTP로 이메일 발송"""
    smtp_server = "smtp.gmail.com"
    smtp_port = 587

    # 메시지 구성
    msg = MIMEMultipart()
    msg['Subject'] = subject
    msg['From'] = sender_email
    msg['To'] = to_email

    # 본문 추가 (텍스트)
    msg.attach(MIMEText(body, 'plain', 'utf-8'))

    try:
        # SMTP 연결 및 발송
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()  # TLS 암호화
            server.login(sender_email, app_password)
            server.send_message(msg)

        print(f"[성공] 이메일 발송 완료: {to_email}")
        return True

    except smtplib.SMTPAuthenticationError:
        print("[오류] 인증 실패 - 이메일 주소와 앱 비밀번호를 확인하세요.")
        return False
    except Exception as e:
        print(f"[오류] 이메일 발송 실패: {e}")
        return False

In [13]:
# 이메일 발송 설정
# .env 파일에서 로드
GMAIL_ADDRESS = os.getenv('GMAIL_ADDRESS')
GMAIL_APP_PASSWORD = os.getenv('GMAIL_APP_PASSWORD')
RECIPIENT_EMAIL = os.getenv('RECIPIENT_EMAIL')  # 수신자

print("[이메일 설정 확인]")
print("=" * 60)
print(f"발신자: {GMAIL_ADDRESS[:10] + '...' if GMAIL_ADDRESS else '미설정'}")
print(f"수신자: {RECIPIENT_EMAIL[:10] + '...' if RECIPIENT_EMAIL else '미설정'}")
print(f"앱 비밀번호: {'설정됨' if GMAIL_APP_PASSWORD else '미설정'}")

[이메일 설정 확인]
발신자: ironmancit...
수신자: youngjea.o...
앱 비밀번호: 설정됨


In [14]:
# 이메일 발송 테스트

if all([GMAIL_ADDRESS, GMAIL_APP_PASSWORD, RECIPIENT_EMAIL]):
    today = datetime.now().strftime('%Y-%m-%d')
    subject = f"[금융 리포트] {today} 오늘의 금융 지표"

    success = send_email_gmail(
        subject=subject,
        body=report,
        to_email=RECIPIENT_EMAIL,
        sender_email=GMAIL_ADDRESS,
        app_password=GMAIL_APP_PASSWORD
    )

    if success:
        print("테스트 이메일 발송 완료!")
else:
    print("이메일 설정이 완료되지 않아 발송을 건너뜁니다.")

[성공] 이메일 발송 완료: youngjea.oh@oyj.ai.kr
테스트 이메일 발송 완료!


---
## 6. 전체 파이프라인 통합

In [15]:
def run_daily_report_pipeline():
    """
    전체 파이프라인 실행:
    1. 데이터 수집 (네이버 금융 + FRED)
    2. SQLite 저장
    3. 리포트 생성
    4. 이메일 발송 (옵션)
    """
    print("[일일 금융 리포트 파이프라인 시작]")
    print("=" * 60)
    start_time = datetime.now()

    # 1. 데이터 수집
    print("\n[1/4] 데이터 수집 중...")

    print("  - 시장 지표 크롤링...")
    df_market = crawl_market_indicators()
    print(f"    -> {len(df_market)}건 수집")

    print("  - 뉴스 크롤링...")
    df_news = crawl_financial_news(limit=5)
    print(f"    -> {len(df_news)}건 수집")

    print("  - FRED 경제지표...")
    df_fred = collect_fred_indicators(FRED_API_KEY)
    print(f"    -> {len(df_fred)}건 수집")

    # 3. 리포트 생성
    print("\n[3/4] 리포트 생성 중...")
    report = generate_report(df_market, df_news, df_fred)

    # 4. 이메일 발송
    if all([GMAIL_ADDRESS, GMAIL_APP_PASSWORD, RECIPIENT_EMAIL]):
        print("\n[4/4] 이메일 발송 중...")
        today_str = datetime.now().strftime('%Y-%m-%d')
        subject = f"[금융 리포트] {today_str} 오늘의 금융 지표"

        success = send_email_gmail(
            subject=subject,
            body=report,
            to_email=RECIPIENT_EMAIL,
            sender_email=GMAIL_ADDRESS,
            app_password=GMAIL_APP_PASSWORD
        )
    else:
        print("\n[4/4] 이메일 발송 건너뜀")

    # 완료
    elapsed = (datetime.now() - start_time).total_seconds()
    print("\n" + "=" * 60)
    print(f"[파이프라인 완료] 소요 시간: {elapsed:.1f}초")

    return {
        'market': df_market,
        'news': df_news,
        'fred': df_fred,
        'report': report
    }

In [16]:
# 파이프라인 실행 (이메일 발송 비활성화)
print("[파이프라인 테스트 실행]")
print("=" * 60)

results = run_daily_report_pipeline()

[파이프라인 테스트 실행]
[일일 금융 리포트 파이프라인 시작]

[1/4] 데이터 수집 중...
  - 시장 지표 크롤링...
    -> 8건 수집
  - 뉴스 크롤링...
    -> 5건 수집
  - FRED 경제지표...
    -> 2건 수집

[3/4] 리포트 생성 중...

[4/4] 이메일 발송 중...
[성공] 이메일 발송 완료: youngjea.oh@oyj.ai.kr

[파이프라인 완료] 소요 시간: 5.9초


In [17]:
# 생성된 리포트 확인
print("\n[생성된 리포트]")
print("=" * 60)
print(results['report'])


[생성된 리포트]
[오늘의 금융 지표 리포트] 2026-01-09

=== 환율 ===
  미국 USD: 1,458.40 (상승 5.40)
  일본 JPY(100엔): 926.59 (상승 0.26)
  유럽연합 EUR: 1,699.33 (상승 5.06)
  중국 CNY: 208.92 (상승 0.82)

=== 원자재 ===
  WTI: 57.76 (상승 1.77)
  휘발유: 1713.62 (하락 2.12)
  국제 금: 4460.7 (하락 1.80)
  국내 금: 209806.68 (상승 1,246.52)

=== 미국 경제지표 ===
  미국 기준금리: 3.72% (기준: 2025-12-01)
  미국 10년 국채: 4.15% (기준: 2026-01-07)

=== 주요 뉴스 ===
  1. 5000피 넘어 6000피 부채질?…'국장 장투' ISA 세제혜택, 효과...
     - 머니투데이
  2. 트럼프 한마디에 이틀 만에 20만원 올랐다…52주 신고가 찍은 한화에어로
     - 매일경제
  3. 국장 특화 '新ISA' 나온다…'역대급' 세혜택 부여
     - 한국경제TV
  4. 최고가 찍었는데 '사라, 마라' 보고서가 없다...한미반도체만 왜?
     - 머니투데이
  5. Private Equity 전략의 핵심, 바이아웃[마켓칼럼]
     - 한국경제

이 리포트는 자동으로 생성되었습니다.


---
## 7. 17차시 스케줄링과 연결

17차시에서 배운 스케줄링 방법을 적용하여 매일 아침 자동 실행합니다.
동일한 작업을 두 가지 방법으로 실행할 수 있습니다.

In [18]:
# 방법 1: Python schedule 라이브러리로 실행
print("\n[방법 1: Python schedule 라이브러리로 자동화]")
print("=" * 60)
print("17차시에서 배운 schedule 라이브러리를 사용합니다.\n")

import schedule
import time

# 실행 횟수 추적 (테스트용)
execution_count = 0

def scheduled_pipeline():
    """스케줄된 파이프라인 실행 (방법 1)"""
    global execution_count
    execution_count += 1
    print(f"\n[실행 #{execution_count}]")
    run_daily_report_pipeline()

# 매일 08:00에 실행하도록 등록 (테스트는 10초마다)
print("스케줄 등록: 10초마다 실행 (테스트용)")
print("실제 사용 시: schedule.every().day.at('08:00').do(scheduled_pipeline)")
schedule.every(10).seconds.do(scheduled_pipeline)

# 첫 번째 즉시 실행
scheduled_pipeline()

# 스케줄 실행 (추가 1회만 - 테스트용)
print("\n다음 실행까지 대기 중...")
while execution_count < 2:
    schedule.run_pending()
    time.sleep(1)

schedule.clear()
print("\n[방법 1 완료] schedule 라이브러리 테스트 종료!")
print("\n[참고] 실제 사용 시:")
print("while True:")
print("    schedule.run_pending()")
print("    time.sleep(60)")


[방법 1: Python schedule 라이브러리로 자동화]
17차시에서 배운 schedule 라이브러리를 사용합니다.

스케줄 등록: 10초마다 실행 (테스트용)
실제 사용 시: schedule.every().day.at('08:00').do(scheduled_pipeline)

[실행 #1]
[일일 금융 리포트 파이프라인 시작]

[1/4] 데이터 수집 중...
  - 시장 지표 크롤링...
    -> 8건 수집
  - 뉴스 크롤링...
    -> 5건 수집
  - FRED 경제지표...
    -> 2건 수집

[3/4] 리포트 생성 중...

[4/4] 이메일 발송 중...
[성공] 이메일 발송 완료: youngjea.oh@oyj.ai.kr

[파이프라인 완료] 소요 시간: 6.0초

다음 실행까지 대기 중...

[실행 #2]
[일일 금융 리포트 파이프라인 시작]

[1/4] 데이터 수집 중...
  - 시장 지표 크롤링...
    -> 8건 수집
  - 뉴스 크롤링...
    -> 5건 수집
  - FRED 경제지표...
    -> 2건 수집

[3/4] 리포트 생성 중...

[4/4] 이메일 발송 중...
[성공] 이메일 발송 완료: youngjea.oh@oyj.ai.kr

[파이프라인 완료] 소요 시간: 5.8초

[방법 1 완료] schedule 라이브러리 테스트 종료!

[참고] 실제 사용 시:
while True:
    schedule.run_pending()
    time.sleep(60)


In [19]:
# 방법 2: Windows 작업 스케줄러용 독립 실행 스크립트 생성
print("\n[방법 2: Windows 작업 스케줄러용 스크립트 생성]")
print("=" * 60)
print("17차시에서 배운 방법으로 독립 실행 스크립트를 생성합니다.\n")

# 동일한 작업을 수행하는 독립 실행 스크립트
standalone_script = '''"""
Windows 작업 스케줄러용 독립 실행 스크립트
18차시 파이프라인을 자동 실행합니다.
방법 1(schedule)과 동일한 작업을 수행합니다.
"""
import sys
from pathlib import Path

# 현재 스크립트 디렉토리를 Python 경로에 추가
script_dir = Path(__file__).parent
sys.path.insert(0, str(script_dir))

# 18차시 모듈의 함수들을 import
try:
    # 필요한 함수들을 import
    import pandas as pd
    import requests
    from bs4 import BeautifulSoup
    from datetime import datetime
    import time
    import os
    from dotenv import load_dotenv
    
    # 환경 변수 로드
    load_dotenv()
    
    # 공통 설정
    HEADERS = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }
    
    def get_soup(url):
        """URL에서 BeautifulSoup 객체 반환"""
        response = requests.get(url, headers=HEADERS, timeout=10)
        response.raise_for_status()
        return BeautifulSoup(response.text, 'html.parser')
    
    def crawl_market_indicators():
        """네이버 금융에서 환율, 유가, 금 시세 크롤링"""
        url = "https://finance.naver.com/marketindex/"
        soup = get_soup(url)
        data = []
        
        def get_direction(item):
            if item.select_one('.point_up'):
                return '상승'
            elif item.select_one('.point_dn'):
                return '하락'
            return '보합'
        
        exchange_list = soup.select('#exchangeList li')
        for item in exchange_list:
            name = item.select_one('.h_lst .blind')
            value = item.select_one('.head_info .value')
            change = item.select_one('.change')
            if name and value:
                data.append({
                    '분류': '환율',
                    '지표명': name.get_text(strip=True),
                    '현재가': value.get_text(strip=True),
                    '등락': change.get_text(strip=True) if change else '',
                    '등락방향': get_direction(item)
                })
        
        oil_gold_list = soup.select('#oilGoldList li')
        for item in oil_gold_list:
            name = item.select_one('.h_lst .blind')
            value = item.select_one('.head_info .value')
            change = item.select_one('.change')
            if name and value:
                data.append({
                    '분류': '원자재',
                    '지표명': name.get_text(strip=True),
                    '현재가': value.get_text(strip=True),
                    '등락': change.get_text(strip=True) if change else '',
                    '등락방향': get_direction(item)
                })
        
        return pd.DataFrame(data) if data else pd.DataFrame()
    
    def crawl_financial_news(limit=5):
        """네이버 금융 주요 뉴스 크롤링"""
        url = "https://finance.naver.com/news/mainnews.naver"
        soup = get_soup(url)
        news_data = []
        news_items = soup.select('ul.newsList li')[:limit]
        
        for item in news_items:
            title_tag = item.select_one('dd.articleSubject a')
            summary_tag = item.select_one('dd.articleSummary')
            press_tag = item.select_one('.press')
            if title_tag:
                title = title_tag.get_text(strip=True)
                link = 'https://finance.naver.com' + title_tag.get('href', '')
                summary = ''
                press = ''
                if summary_tag:
                    summary_text = summary_tag.get_text(strip=True)
                    if press_tag:
                        press = press_tag.get_text(strip=True)
                        summary = summary_text.replace(press, '').strip()
                news_data.append({
                    '제목': title,
                    '요약': summary[:100] + '...' if len(summary) > 100 else summary,
                    '출처': press,
                    '링크': link
                })
        return pd.DataFrame(news_data) if news_data else pd.DataFrame()
    
    def fetch_fred_series(series_id, api_key):
        """FRED API에서 단일 시리즈 데이터 조회"""
        if not api_key:
            return None
        base_url = "https://api.stlouisfed.org/fred/series/observations"
        params = {
            'series_id': series_id,
            'api_key': api_key,
            'file_type': 'json',
            'sort_order': 'desc',
            'limit': 1
        }
        try:
            response = requests.get(base_url, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()
            observations = data.get('observations', [])
            if observations:
                obs = observations[0]
                return {
                    'series_id': series_id,
                    'date': obs['date'],
                    'value': obs['value']
                }
        except Exception as e:
            print(f"  [오류] {series_id}: {e}")
        return None
    
    def collect_fred_indicators(api_key):
        """주요 FRED 경제지표 수집"""
        if not api_key:
            return pd.DataFrame()
        indicators = {'FEDFUNDS': '미국 기준금리', 'DGS10': '미국 10년 국채'}
        data = []
        for series_id, name in indicators.items():
            result = fetch_fred_series(series_id, api_key)
            if result:
                data.append({
                    '분류': '경제지표',
                    '지표명': name,
                    '현재가': f"{result['value']}%",
                    '기준일': result['date']
                })
            time.sleep(0.3)
        return pd.DataFrame(data) if data else pd.DataFrame()
    
    def generate_report(df_market, df_news, df_fred):
        """리포트 생성"""
        today = datetime.now().strftime('%Y-%m-%d')
        lines = []
        lines.append(f"[오늘의 금융 지표 리포트] {today}")
        lines.append("=" * 50)
        
        lines.append("\\n=== 환율 ===")
        df_exchange = df_market[df_market['분류'] == '환율']
        for _, row in df_exchange.iterrows():
            direction = f"({row['등락방향']} {row['등락']})" if row['등락'] else ""
            lines.append(f"  {row['지표명']}: {row['현재가']} {direction}")
        
        lines.append("\\n=== 원자재 ===")
        df_commodity = df_market[df_market['분류'] == '원자재']
        for _, row in df_commodity.iterrows():
            direction = f"({row['등락방향']} {row['등락']})" if row['등락'] else ""
            lines.append(f"  {row['지표명']}: {row['현재가']} {direction}")
        
        lines.append("\\n=== 미국 경제지표 ===")
        for _, row in df_fred.iterrows():
            lines.append(f"  {row['지표명']}: {row['현재가']} (기준: {row['기준일']})")
        
        lines.append("\\n=== 주요 뉴스 ===")
        for i, row in df_news.iterrows():
            title = row['제목'][:40] + '...' if len(row['제목']) > 40 else row['제목']
            lines.append(f"  {i+1}. {title}")
            lines.append(f"     - {row['출처']}")
        
        lines.append("\\n" + "=" * 50)
        lines.append("이 리포트는 자동으로 생성되었습니다.")
        return "\\n".join(lines)
    
    def run_daily_report_pipeline():
        """전체 파이프라인 실행 (방법 1과 동일)"""
        print("[일일 금융 리포트 파이프라인 시작]")
        print("=" * 60)
        start_time = datetime.now()
        
        print("\\n[1/4] 데이터 수집 중...")
        print("  - 시장 지표 크롤링...")
        df_market = crawl_market_indicators()
        print(f"    -> {len(df_market)}건 수집")
        
        print("  - 뉴스 크롤링...")
        df_news = crawl_financial_news(limit=5)
        print(f"    -> {len(df_news)}건 수집")
        
        print("  - FRED 경제지표...")
        FRED_API_KEY = os.getenv('FRED_API_KEY')
        df_fred = collect_fred_indicators(FRED_API_KEY)
        print(f"    -> {len(df_fred)}건 수집")
        
        print("\\n[2/4] 리포트 생성 중...")
        report = generate_report(df_market, df_news, df_fred)
        
        # 리포트를 파일로 저장
        today_str = datetime.now().strftime('%Y%m%d')
        report_file = script_dir / f"report_{today_str}.txt"
        with open(report_file, 'w', encoding='utf-8') as f:
            f.write(report)
        print(f"    -> 리포트 저장: {report_file}")
        
        # 이메일 발송 (옵션)
        GMAIL_ADDRESS = os.getenv('GMAIL_ADDRESS')
        GMAIL_APP_PASSWORD = os.getenv('GMAIL_APP_PASSWORD')
        RECIPIENT_EMAIL = os.getenv('RECIPIENT_EMAIL')
        
        if all([GMAIL_ADDRESS, GMAIL_APP_PASSWORD, RECIPIENT_EMAIL]):
            print("\\n[3/4] 이메일 발송 중...")
            import smtplib
            from email.mime.text import MIMEText
            from email.mime.multipart import MIMEMultipart
            
            smtp_server = "smtp.gmail.com"
            smtp_port = 587
            msg = MIMEMultipart()
            msg['Subject'] = f"[금융 리포트] {today_str} 오늘의 금융 지표"
            msg['From'] = GMAIL_ADDRESS
            msg['To'] = RECIPIENT_EMAIL
            msg.attach(MIMEText(report, 'plain', 'utf-8'))
            
            try:
                with smtplib.SMTP(smtp_server, smtp_port) as server:
                    server.starttls()
                    server.login(GMAIL_ADDRESS, GMAIL_APP_PASSWORD)
                    server.send_message(msg)
                print(f"    -> 이메일 발송 완료: {RECIPIENT_EMAIL}")
            except Exception as e:
                print(f"    -> 이메일 발송 실패: {e}")
        else:
            print("\\n[3/4] 이메일 발송 건너뜀 (설정 없음)")
        
        elapsed = (datetime.now() - start_time).total_seconds()
        print("\\n" + "=" * 60)
        print(f"[파이프라인 완료] 소요 시간: {elapsed:.1f}초")
    
    # 파이프라인 실행
    run_daily_report_pipeline()
    sys.exit(0)
    
except ImportError as e:
    print(f"[오류] 모듈 import 실패: {e}")
    print("필요한 라이브러리가 설치되어 있는지 확인하세요.")
    sys.exit(1)
except Exception as e:
    print(f"[오류] 실행 실패: {e}")
    import traceback
    traceback.print_exc()
    sys.exit(1)
'''

print(standalone_script)


[방법 2: Windows 작업 스케줄러용 스크립트 생성]
17차시에서 배운 방법으로 독립 실행 스크립트를 생성합니다.

"""
Windows 작업 스케줄러용 독립 실행 스크립트
18차시 파이프라인을 자동 실행합니다.
방법 1(schedule)과 동일한 작업을 수행합니다.
"""
import sys
from pathlib import Path

# 현재 스크립트 디렉토리를 Python 경로에 추가
script_dir = Path(__file__).parent
sys.path.insert(0, str(script_dir))

# 18차시 모듈의 함수들을 import
try:
    # 필요한 함수들을 import
    import pandas as pd
    import requests
    from bs4 import BeautifulSoup
    from datetime import datetime
    import time
    import os
    from dotenv import load_dotenv
    
    # 환경 변수 로드
    load_dotenv()
    
    # 공통 설정
    HEADERS = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }
    
    def get_soup(url):
        """URL에서 BeautifulSoup 객체 반환"""
        response = requests.get(url, headers=HEADERS, timeout=10)
        response.raise_for_status()
        return BeautifulSoup(response.text, 'html.parser')
    
    def crawl_market_indicato

In [20]:
# 독립 실행 스크립트 파일 저장
with open('daily_finance_report.py', 'w', encoding='utf-8') as f:
    f.write(standalone_script)
print("\ndaily_finance_report.py 파일 생성 완료!")


daily_finance_report.py 파일 생성 완료!


---
## 학습 정리

### 모듈 2 완료
이것으로 "경제 금융 지표 수집 자동화" 모듈이 완료되었습니다.

- 11차시: API 개념 (DART, FRED)
- 12차시: DART API로 공시정보 수집
- 13차시: FRED API로 경제지표 수집
- 14차시: 웹 크롤링 기초
- 15차시: 네이버 금융 크롤링
- 16차시: 데이터 SQLite 저장
- 17차시: 자동화 스케줄링
- 18차시: 통합 파이프라인 및 이메일 발송